In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from xgboost import XGBRegressor

In [17]:
df = pd.read_csv('data/data-ready.csv')

targets = ["z1", "z2", "z3"]

df.drop("DateTime", inplace=True, axis=1)

# split train and test
X = df.drop(columns=targets, axis=1)
y1 = df['z1']
X1_train, X1_test, y1_train, y1_test = train_test_split(X, y1, test_size=0.2, random_state=42)

y2 = df['z2']
X2_train, X2_test, y2_train, y2_test = train_test_split(X, y2, test_size=0.2, random_state=42)

y3 = df['z3']
X3_train, X3_test, y3_train, y3_test = train_test_split(X, y3, test_size=0.2, random_state=42)

In [19]:
from sklearn.model_selection import RepeatedKFold, cross_val_score

# fit an xgboost model to the data
model = XGBRegressor()
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X1_train, y1_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

# force scores to be positive
scores = np.abs(scores)
print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()))

Mean MAE: 813.266 (12.516)


In [10]:
X1_train.describe()

,Temperature,Humidity,Wind Speed,general diffuse flows,diffuse flows,Year,Month,Day,Hour,IsWeekend,IsNight,HourCos,HourSin,MonthCos,MonthSin,DayCos,DaySin
count,4.193200e+04,4.193200e+04,4.193200e+04,4.193200e+04,4.193200e+04,41932.0,4.193200e+04,4.193200e+04,4.193200e+04,4.193200e+04,4.193200e+04,4.193200e+04,4.193200e+04,4.193200e+04,4.193200e+04,4.193200e+04,4.193200e+04
mean,-2.033414e-18,-1.499643e-17,1.580132e-17,3.948213e-17,3.329716e-17,0.0,-2.541768e-17,-2.812890e-17,3.524585e-17,5.185206e-17,7.210148e-17,-2.270646e-17,-2.101195e-17,1.880908e-17,2.041887e-17,2.143558e-17,2.024942e-17
std,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00,0.0,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00
min,-2.686307e+00,-3.666732e+00,-8.123709e-01,-6.936579e-01,-6.054479e-01,0.0,-1.603045e+00,-1.671973e+00,-1.663930e+00,-6.324714e-01,-9.184607e-01,-1.409936e+00,-1.417482e+00,-1.404274e+00,-1.408389e+00,-1.390588e+00,-1.401267e+00
25%,-7.565154e-01,-6.421023e-01,-8.004477e-01,-6.934387e-01,-6.045541e-01,0.0,-7.306391e-01,-8.743884e-01,-7.952920e-01,-6.324714e-01,-9.184607e-01,-9.956988e-01,-1.003286e+00,-1.214990e+00,-1.218723e+00,-1.053252e+00,-1.017559e+00
50%,-6.820076e-04,1.026201e-01,-7.970411e-01,-6.719039e-01,-5.647570e-01,0.0,1.417670e-01,3.713696e-02,-7.142672e-02,-6.324714e-01,-9.184607e-01,4.357145e-03,-3.326490e-03,8.560384e-03,7.297759e-03,-4.156480e-02,-2.279819e-03
75%,7.016564e-01,8.466983e-01,1.259282e+00,5.266347e-01,2.101334e-01,0.0,1.014173e+00,8.347216e-01,7.972116e-01,1.581099e+00,1.088778e+00,1.004413e+00,9.966326e-01,7.149774e-01,7.151412e-01,1.015137e+00,1.012999e+00
max,3.618000e+00,1.709958e+00,1.926980e+00,3.700190e+00,6.931156e+00,0.0,1.595777e+00,1.746247e+00,1.665850e+00,1.581099e+00,1.088778e+00,1.418650e+00,1.410829e+00,1.421395e+00,1.422985e+00,1.459515e+00,1.396708e+00
